In [1]:
import psycopg2
import openai
from utils import *
import logging 
from log_config import configure_logging
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [ ]:
MODEL_GPT = "gpt-4o-mini"

MODEL_DS = "deepseek-chat"

DATABASE_CONFIG = {
    "dbname": "tensorflow_data",
    "user": "postgres",
    "password": "YzLq4UAMQ7LOadXh",
    "host": "humanely-winged-bluebird.data-1.use1.tembo.io",
    "port": 5432,
}

conn = psycopg2.connect(**DATABASE_CONFIG)
cursor = conn.cursor()

configure_logging()

In [3]:
cursor.execute(
    """
    SELECT 
        issue_id,
        title,
        body
    FROM   
        issues_from_release
    WHERE
        release_number LIKE '%2.17%'
    """
)
issues = cursor.fetchall()
len(issues)

405

In [ ]:
client_gpt = openai.OpenAI(api_key=OPENAI_API_KEY)
client_ds = openai.OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

In [5]:
for issue in issues:
    issue_id, issue_title, issue_body = issue
    
    try:
        comments = get_comments_by_issue_id(issue_id, cursor)
        logging.info(f"{len(comments)} comments found for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when getting the comments for issue {issue_id}: {e}")
        continue

    try:
        logging.info(f"Starting sentiment analysis with gpt-4o-mini for issue {issue_id}")
        sentiments_gpt = analyze_issue_sentiment_openai(
            issue_title=issue_title,
            issue_body=issue_body,
            comments=comments,
            client=client_gpt,
            model=MODEL_GPT
        )
        logging.info(f"Sentiment analysis with gpt-4o-mini concluded for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when analyzing issue {issue_id} with gpt-4o-mini.")
        continue
    
    """
    try:
        logging.info(f"Starting sentiment analysis with deepseek-v3 for issue {issue_id}")
        sentiments_ds = analyze_issue_sentiment_openai(
            issue_title=issue_title,
            issue_body=issue_body,
            comments=comments,
            client=client_ds,
            model=MODEL_DS
        )
        logging.info(f"Sentiment analysis with deepseek-v3 concluded for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when analyzing issue {issue_id} with gpt-4o-mini.")
        continue
    """

    try:
        save_sentiments_gpt(sentiments_gpt, cursor)
        # save_sentiments_ds(sentiments_ds, cursor)
        logging.info(f"Data successfully saved for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when saving the data for issue {issue_id}.")
        continue

2025-04-24 12:36:46,293 - INFO - 12 comments found for issue 2719667595.
2025-04-24 12:36:46,294 - INFO - Starting sentiment analysis with gpt-4o-mini for issue 2719667595
2025-04-24 12:36:46,295 - INFO - Classifying comment 2519970094...
2025-04-24 12:36:47,766 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-24 12:36:47,771 - INFO - Sentiment of the comment is: neutral
2025-04-24 12:36:47,772 - INFO - Classifying comment 2521726106...
2025-04-24 12:36:48,625 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-24 12:36:48,706 - INFO - Sentiment of the comment is: negative
2025-04-24 12:36:48,706 - INFO - Classifying comment 2527402162...
2025-04-24 12:36:49,630 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-24 12:36:49,699 - INFO - Sentiment of the comment is: positive
2025-04-24 12:36:49,700 - INFO - Classifying comment 2531488173...
2025-04-2

In [6]:
conn.commit()
cursor.close()  
conn.close() 